In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [2]:
data = pd.read_csv('EEG_Eye_State.csv')
# data=data.drop("eyeDetection",axis=1)

data

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,0
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,0
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14975,4281.03,3990.26,4245.64,4116.92,4333.85,4614.36,4074.87,4625.64,4203.08,4221.54,4171.28,4269.23,4593.33,4340.51,1
14976,4276.92,3991.79,4245.13,4110.77,4332.82,4615.38,4073.33,4621.54,4194.36,4217.44,4162.56,4259.49,4590.26,4333.33,1
14977,4277.44,3990.77,4246.67,4113.85,4333.33,4615.38,4072.82,4623.59,4193.33,4212.82,4160.51,4257.95,4591.79,4339.49,1
14978,4284.62,3991.79,4251.28,4122.05,4334.36,4616.41,4080.51,4628.72,4200.00,4220.00,4165.64,4267.18,4596.41,4350.77,1


In [3]:
import pandas as pd

# Create a DataFrame (assuming you already have this step)
data = pd.DataFrame(data)

def remove_outliers_iqr(data, column, eye_detection_column):
    # Ensure the columns are numeric
    data[column] = pd.to_numeric(data[column], errors='coerce')
    eye_detection_column = pd.to_numeric(eye_detection_column, errors='coerce')

    # Calculate the first quartile and the third quartile
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    # Calculate the IQR
    iqr = q3 - q1
    # Calculating the threshold
    threshold = 3 * iqr
    # Defining the lower and upper bounds
    lower_bound = q1 - threshold
    upper_bound = q3 + threshold
    # Filtering and returning the filtered data and eyeDetection column
    filtered_data = data[(data[column] >= lower_bound) &
                         (data[column] <= upper_bound)]
    filtered_eye_detection = eye_detection_column[(data[column] >= lower_bound) &
                                                  (data[column] <= upper_bound)]
    return filtered_data, filtered_eye_detection

# Specify the name of your "eyeDetection" column
eye_detection_column = data["eyeDetection"]

# Let's apply our modified method and save the new filtered data and eyeDetection column
for q in data.columns:
    data, eye_detection_column = remove_outliers_iqr(data, q, eye_detection_column)

# Taking a look at our filtered data
print(data)

# Taking a look at the filtered "eyeDetection" column
print(eye_detection_column)


           AF3       F7       F3      FC5       T7       P7       O1       O2  \
0      4329.23  4009.23  4289.23  4148.21  4350.26  4586.15  4096.92  4641.03   
1      4324.62  4004.62  4293.85  4148.72  4342.05  4586.67  4097.44  4638.97   
2      4327.69  4006.67  4295.38  4156.41  4336.92  4583.59  4096.92  4630.26   
3      4328.72  4011.79  4296.41  4155.90  4343.59  4582.56  4097.44  4630.77   
4      4326.15  4011.79  4292.31  4151.28  4347.69  4586.67  4095.90  4627.69   
...        ...      ...      ...      ...      ...      ...      ...      ...   
14975  4281.03  3990.26  4245.64  4116.92  4333.85  4614.36  4074.87  4625.64   
14976  4276.92  3991.79  4245.13  4110.77  4332.82  4615.38  4073.33  4621.54   
14977  4277.44  3990.77  4246.67  4113.85  4333.33  4615.38  4072.82  4623.59   
14978  4284.62  3991.79  4251.28  4122.05  4334.36  4616.41  4080.51  4628.72   
14979  4287.69  3997.44  4260.00  4121.03  4333.33  4616.41  4088.72  4638.46   

            P8       T8    

In [4]:
import numpy as np

# Select all columns except 'eyeDetection'
eeg_data = data.loc[:, data.columns != 'eyeDetection']

# Count the number of rows in the selected data
count = eeg_data.shape[0]

# Calculate the time interval
time_interval = round(117 / count, 5)

# Create the x_axis using linspace
x_axis = np.linspace(0, 117, count)


In [5]:
from scipy.signal import butter, lfilter

# Assuming you have a DataFrame named 'data'
count = data.count(axis=0).values[0]
total_interval = 117

sample_period = total_interval / count

f_sample = 1 / sample_period

# Define a bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

# Create a list of columns to filter, excluding 'eyeDetection'
columns_to_filter = [col for col in data.columns if col != 'eyeDetection']

# Apply bandpass filter to all selected columns
for column in columns_to_filter:
    data[column + '_filtered'] = bandpass_filter(data[column], 3, 20, f_sample, 3)

# Now, you have filtered columns for all data except 'eyeDetection'.



In [6]:
df = pd.DataFrame(data)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assuming that your DataFrame contains the "eyeDetection" column, you can continue with the following steps:

# Separate features (X) and target (y)
X = df.drop("eyeDetection", axis=1)  # Features
y = df['eyeDetection']  # Target variable

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


In [7]:
from sklearn.preprocessing import LabelEncoder

# Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)


# Create and train a Random Forest classifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
print(y_pred)
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy * 100:0.2f}%")
print(report)

<built-in method all of numpy.ndarray object at 0x0000013CB54E4690>
Model Accuracy: 90.35%
              precision    recall  f1-score   support

           0       0.89      0.94      0.91      2273
           1       0.92      0.86      0.89      1840

    accuracy                           0.90      4113
   macro avg       0.91      0.90      0.90      4113
weighted avg       0.90      0.90      0.90      4113

